In [1]:
# Cell 1: Imports and device check
import os, glob
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda:0


In [7]:
class DummyImageDataset(Dataset):
    def __init__(self, data_folder):
        # Gather all .pt files
        self.files = sorted(glob.glob(os.path.join(data_folder, 'sample_*.pt')))
    def __len__(self):
        return len(self.files)
    def __getitem__(self, idx):
        d = torch.load(self.files[idx], map_location='cpu')
        img = d['img'].float()         # C x H x W
        # Assign a deterministic dummy label: e.g. even idx=0, odd=1
        label = torch.tensor(idx % 2, dtype=torch.long)
        return img, label

In [9]:
data_folder = '/home/exouser/pose_estim/clipestimator/data/synthetic_data/'  # adjust path if needed
dataset = DummyImageDataset(data_folder)
loader  = DataLoader(dataset, batch_size=64, shuffle=True, pin_memory=True)
print(f"Dataset size: {len(dataset)} samples")

Dataset size: 10 samples


In [10]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.pool  = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.fc1   = nn.Linear(32 * 64 * 64, 64)
        self.fc2   = nn.Linear(64, 2)
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))  # 3x256x256 -> 16x128x128
        x = self.pool(F.relu(self.conv2(x)))  # 16x128x128 -> 32x64x64
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = SimpleCNN().to(device)
print(model)

SimpleCNN(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=131072, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=2, bias=True)
)


In [11]:
otimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()

epochs = 3
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for imgs, labels in loader:
        # Transfer to GPU asynchronously
        imgs   = imgs.to(device, non_blocking=True)
        labels = labels.to(device, non_blocking=True)
        # Forward + backward
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * imgs.size(0)
    epoch_loss = running_loss / len(dataset)
    print(f"Epoch {epoch+1}/{epochs}, Loss: {epoch_loss:.4f}")

# Cell 6: Verify GPU Utilization
# During training, monitor nvidia-smi; this loop should engage the GPU.
print("Training complete. Please check nvidia-smi to confirm GPU usage.")

/home/exouser/anaconda3/envs/blvenv/lib/python3.9/site-packages/torch/nn/modules/conv.py:456: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at /opt/conda/conda-bld/pytorch_1716905971093/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:84.)
  return F.conv2d(input, weight, bias, self.stride,


NameError: name 'optimizer' is not defined